In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
base  = pd.read_csv(r'F:\study\ml\DataSet\beijinPM25\PRSA_data_2010.1.1-2014.12.31.csv')
base.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [3]:
base = base.dropna()

In [4]:
base = base.drop('No', axis = 1)
base = base.drop('year', axis = 1)
base = base.drop('month', axis = 1)
base = base.drop('day', axis = 1)
base = base.drop('hour', axis = 1)
base = base.drop('cbwd', axis = 1)

In [5]:
base.head()

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir
24,129.0,-16,-4.0,1020.0,1.79,0,0
25,148.0,-15,-4.0,1020.0,2.68,0,0
26,159.0,-11,-5.0,1021.0,3.57,0,0
27,181.0,-7,-5.0,1022.0,5.36,1,0
28,138.0,-7,-5.0,1022.0,6.25,2,0


In [6]:
base_treinamento = base.iloc[:, 1:7].values

In [7]:
poluicao = base.iloc[:, 0].values

In [12]:
poluicao.shape

(41757, 1)

In [8]:
from sklearn.preprocessing import  MinMaxScaler

In [9]:
normalizador = MinMaxScaler(feature_range = (0, 1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [10]:
poluicao = poluicao.reshape(-1, 1)
poluicao_normalizado = normalizador.fit_transform(poluicao)

In [13]:
previsores = []
poluicao_real = []
for i in range(10, 41757):
    previsores.append(base_treinamento_normalizada[i-10:i, 0:6])
    poluicao_real.append(poluicao_normalizado[i, 0])
previsores, poluicao_real = np.array(previsores), np.array(poluicao_real)

In [14]:
previsores.shape,poluicao_real.shape

((41747, 10, 6), (41747,))

In [21]:
class Extractlastcell(nn.Module):
    def forward(self,x):
        out,_=x
        return out[:,-1,:]

In [22]:
previsores[:,-1,:].shape

(41747, 6)

In [30]:

net = nn.Sequential(nn.LSTM(60, 100),print('---'),
                    nn.Dropout(0.3), nn.LSTM(100, 50),
                    nn.Dropout(0.2), nn.LSTM(50, 50), nn.Dropout(0.2),
                    nn.LSTM(50, 50), nn.Dropout(0.2),Extractlastcell(),nn.Linear(50,1))

---


In [50]:
net.state_dict

<bound method Module.state_dict of Sequential(
  (0): LSTM(6, 100)
  (1): None
  (2): Extractlastcell()
  (3): Linear(in_features=50, out_features=1, bias=True)
)>

In [46]:
net = nn.Sequential(nn.LSTM(6, 100), print('---'), Extractlastcell(),
                    nn.Linear(50, 1))

---


In [ ]:
class SelectItem(nn.Module):
    def __init__(self, item_index):
        super(SelectItem, self).__init__()
        self._name = 'selectitem'
        self.item_index = item_index

    def forward(self, inputs):
        return inputs[self.item_index]

In [47]:
def train_pytorch(net,num_epochs,lr,train_features,train_labels,batch_size):
#     net=LSTM_Reg(162,256,2,1)
    loss=nn.L1Loss()
    optimizer=optim.Adam(net.parameters(),lr=5e-3)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True,drop_last=True)
    state=None
    for e in range(num_epochs):
        l_sum=[]
        net.train()
        for x,y in data_iter:
            if state is not None:
                if isinstance(state ,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
                
#             x=x.unsqueeze(dim=0)
            output=net(x)
            l=loss(output,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            l_sum.append(l.item())
            
        if (e+1) % 50 ==0:
            net.eval()
            with torch.no_grad():
                print('epoch %d , train loss mean : %f,train loss std : %f' % ( e+1,np.mean(l_sum),np.std(l_sum) ))

In [48]:
x9.shape

torch.Size([41747, 10, 6])

In [49]:
x9=torch.Tensor(previsores)
y9=torch.Tensor(poluicao_real)
train_pytorch(net,100,0.001,x9,y9,64)

TypeError: 'NoneType' object is not callable

In [15]:
class LSTM_Reg(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,layer_size=1):
        super().__init__()
        self.lstm_net1=nn.LSTM(input_size=input_size,hidden_size=100,num_layers=2,dropout=0.3)
        self.lstm_net2=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=2,dropout=0.2)
        self.lstm_net3=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=2,dropout=0.2)
        self.lstm_net4=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=2,dropout=0.2)
        self.dense1=nn.Linear(100,50)
        self.dense2=nn.Linear(50,1)
        self.state=None
    
    def forward(self,x,state):
#         x=x.reshape(x.shape[0],-1)
        y,self.state=self.lstm_net1(x,state)
        y=self.dense1(y.reshape(-1,y.shape[-1]))
        self.state[0]=self.dense1(self.state[0].reshape(-1,self.state[0].shape[-1]))
        self.state[1]=self.dense1(self.state[1].reshape(-1,self.state[1].shape[-1]))
        y=y.unsqueeze(dim=0)
        self.state[0]=self.state[0].unsqueeze(dim=0)
        self.state[1]=self.state[1].unsqueeze(dim=0)
        y,self.state=self.lstm_net2(y,self.state)
        output=self.dense2(y.view(-1,y.shape[-1]))
        return output,self.state

In [17]:
net1=LSTM_Reg(1320,100,1,1)
net1.state_dict

D:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
D:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<bound method Module.state_dict of LSTM_Reg(
  (lstm_net1): LSTM(1320, 100, dropout=0.3)
  (lstm_net2): LSTM(1320, 50, dropout=0.2)
  (lstm_net3): LSTM(1320, 50, dropout=0.2)
  (lstm_net4): LSTM(1320, 50, dropout=0.2)
  (dense1): Linear(in_features=100, out_features=50, bias=True)
  (dense2): Linear(in_features=50, out_features=1, bias=True)
)>

In [ ]:
    >>> rnn = nn.LSTM(10, 20, 2)
    >>> input = torch.randn(5, 3, 10)
    >>> h0 = torch.randn(2, 3, 20)
    >>> c0 = torch.randn(2, 3, 20)
    >>> output, (hn, cn) = rnn(input, (h0, c0))
Init docstring: Initializes internal Module state,

In [61]:
net = nn.Sequential(nn.LSTM(6, 100), print('---'), Extractlastcell(),
                    nn.Linear(100, 1))

---


In [56]:
input = torch.randn(5, 3, 6)
rnn = nn.LSTM(6, 100, 2)
output,(hn,cn)=rnn(input,(None))

In [58]:
output.shape

torch.Size([5, 3, 100])

In [60]:
hn.shape,cn.shape

(torch.Size([2, 3, 100]), torch.Size([2, 3, 100]))

In [99]:
class SelectItem(nn.Module):
    def __init__(self, item_index):
        super(SelectItem, self).__init__()
        self._name = 'selectitem'
        self.item_index = item_index
        print('---')

    def forward(self, inputs,state):
        print('call forward !')
        return inputs

In [100]:
net = nn.Sequential(nn.LSTM(6, 100))

In [107]:
class ExtractInput(nn.Module):
    def forward(self,x,state):
        output,_=x,state
        print('output.shape : ',output.shape)
        return output

In [117]:
net = nn.Sequential(nn.LSTM(6, 100),ExtractInput(),nn.LSTM(100, 50))

In [118]:
input = torch.randn(5, 3, 6)
output,(hn,cn)=net(input)
# rnn = nn.LSTM(6, 100, 2)
# output=net(input,(None))

output.shape :  torch.Size([5, 3, 100])


In [119]:
output.shape,hn.shape,cn.shape

(torch.Size([5, 3, 50]), torch.Size([1, 3, 50]), torch.Size([1, 3, 50]))